<h2>Deep learning model for DNA sequence function prediction</h2>

In [39]:
import re
import pandas as pd
import numpy as np
from collections import Counter
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall
from sklearn.model_selection import train_test_split

<h3>Data extraction, preprocessing and visualization</h3>

In [24]:
f = open("dna_70k_balanced.txt", "r")
t = f.read()
f.close()

full_sequences = re.findall("([ATGC]+)\s+(\d+)", t)
sequences = [s.replace("\n","") for s, c in full_sequences]
classes = [c for s, c in full_sequences]

In [37]:
dataset = {
    "Class Number": [0, 1, 2, 3, 4, 5, 6],
    "Class Name": [
        "G protein coupled receptors", 
        "Tyrosine kinase", 
        "Tyrosine phosphatase", 
        "Synthetase", 
        "Synthase", 
        "Ion channel", 
        "Transcription factor"
    ],
    "Number of sequences": [9920, 10000, 9920, 9800, 9600, 11100, 10100]
}

df = pd.DataFrame(dataset)

styled_df = df.style.set_table_styles(
    [{
        'selector': 'th',
        'props': [('text-align', 'center')]  
    }],
    overwrite=False
).set_properties(**{
    'text-align': 'left',  
    'border-color': 'black',
    'border-style' :'solid',
    'border-width': '1px'
}).set_caption("Information about the dataset")  

styled_df

,Class Number,Class Name,Number of sequences
0,0,G protein coupled receptors,9920
1,1,Tyrosine kinase,10000
2,2,Tyrosine phosphatase,9920
3,3,Synthetase,9800
4,4,Synthase,9600
5,5,Ion channel,11100
6,6,Transcription factor,10100


<h3>Data encoding and padding</h3>

In [3]:
max_length = max(len(sequence) for sequence in sequences)

In [4]:
def encode_pad_sequences(dna_sequences, max_sequence_length):
    
    molar_mass_values = {'A': 135, 'C': 111, 'G': 151, 'T': 126} #approximate molar mass values
    
    encoded_sequences = np.zeros((len(dna_sequences), max_sequence_length), dtype=int)

    for i, sequence in enumerate(dna_sequences):
        #padding with 0 (without truncating longer sequences to avoid data loss)
        encoded_sequence = [molar_mass_values.get(base, 0) for base in sequence]
        encoded_sequences[i, :len(encoded_sequence)] = encoded_sequence 

    return encoded_sequences

In [5]:
sequences_encoded = encode_pad_sequences(sequences, max_length)

In [6]:
sequences_encoded.shape

(70440, 7536)

In [7]:
classes_np = np.array(classes, dtype='int')
classes_encoded = to_categorical(classes_np, num_classes=7)

In [8]:
classes_encoded.shape

(70440, 7)

<h3>Data splitting for model training</h3>

In [9]:
X_train, X_temp, y_train, y_temp = train_test_split(sequences_encoded, classes_encoded, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [10]:
X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_val_reshaped = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))
X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

<h3>Neural network model</h3>

In [13]:
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

In [11]:
model = Sequential()

# First Convolutional Layer
model.add(Conv1D(filters=64, kernel_size=5, activation='relu', input_shape=(max_length, 1)))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))

# Second Convolutional Layer
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))

# GRU Layer
model.add(GRU(64, return_sequences=False))  
model.add(Dropout(0.4))

# Fully Connected Layers
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.4))

# Output Layer
model.add(Dense(7, activation='softmax'))

In [12]:
optimizer = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy', Precision(), Recall()])

<h3>Model training</h3>

In [14]:
lr_scheduler = LearningRateScheduler(scheduler)

In [15]:
history = model.fit(X_train_reshaped, y_train, validation_data=(X_val_reshaped, y_val), epochs=20, batch_size=64, callbacks=[lr_scheduler])

Epoch 1/20
771/771 [==============================] - 1923s 2s/step - loss: 1.9432 - accuracy: 0.1550 - precision: 0.8079 - recall: 0.0029 - val_loss: 1.9358 - val_accuracy: 0.1602 - val_precision: 1.0000 - val_recall: 0.0028 - lr: 0.0010
Epoch 2/20
771/771 [==============================] - 1921s 2s/step - loss: 1.9209 - accuracy: 0.1691 - precision: 0.9320 - recall: 0.0131 - val_loss: 1.9116 - val_accuracy: 0.1321 - val_precision: 0.5635 - val_recall: 0.0453 - lr: 0.0010
Epoch 3/20
771/771 [==============================] - 1925s 2s/step - loss: 1.9033 - accuracy: 0.1776 - precision: 0.7998 - recall: 0.0199 - val_loss: 1.9168 - val_accuracy: 0.1608 - val_precision: 1.0000 - val_recall: 0.0136 - lr: 0.0010
Epoch 4/20
771/771 [==============================] - 1922s 2s/step - loss: 1.8766 - accuracy: 0.1890 - precision: 0.8361 - recall: 0.0283 - val_loss: 2.3820 - val_accuracy: 0.1475 - val_precision: 0.4704 - val_recall: 0.0738 - lr: 0.0010
Epoch 5/20
771/771 [========================

<h3>Model evaluation</h3>

In [18]:
test_loss, test_accuracy, test_precision, test_recall = model.evaluate(X_test_reshaped, y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}, Test Precision: {test_precision}, Test Recall: {test_recall}")

331/331 [==============================] - 200s 605ms/step - loss: 0.4042 - accuracy: 0.8233 - precision: 0.9819 - recall: 0.6794
Test Loss: 0.40415069460868835, Test Accuracy: 0.8233011364936829, Test Precision: 0.9819450378417969, Test Recall: 0.6794434785842896
